# NLE Assessed Coursework 3: Question 1

For this assessment, you are expected to complete and submit 4 notebook files.  There is 1 notebook file for each question (to speed up load times).  This is notebook 1 out of 4.

Marking guidelines are provided as a separate document.

In order to provide unique datasets for analysis by different students, you must enter your candidate number in the following cell.

In [1]:
candidateno=215921 #this MUST be updated to your candidate number so that you get a unique data sample


In [7]:
#preliminary imports
import sys
sys.path.append(r'\\ad.susx.ac.uk\ITS\TeachingResources\Departments\Informatics\LanguageEngineering\resources')
sys.path.append(r'/Users/talahaddadin/Desktop/Year 2/Semester 1/Natural Language Engineering/resources')

import re
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from itertools import zip_longest
from nltk.tokenize import word_tokenize

from sussex_nltk.corpus_readers import AmazonReviewCorpusReader
import random
from nltk.corpus import stopwords

import spacy
nlp=spacy.load('en')
from nltk.corpus import gutenberg

SyntaxError: invalid syntax (<ipython-input-7-cb43191075d4>, line 16)

## Question 1: Part-of-Speech Tagging (25 marks)


In [6]:
#Do NOT change the code in this cell.

#preparing corpus
def display_sent(asent):
    headings=["token","lower","lemma","pos","NER"]
    info=[]
    for t in asent:
        info.append([t.text,t.lower_,t.lemma_,t.pos_,t.ent_type_])
    return(pd.DataFrame(info,columns=headings))

def tag_sent(asent):
    tagged=[]
    for t in asent:
        tagged.append((t.lower_,t.pos_))
    return tagged

def clean_text(astring):
    #replace newlines with space
    newstring=re.sub("\n"," ",astring)
    #remove title and chapter headings
    newstring=re.sub("\[[^\]]*\]"," ",newstring)
    newstring=re.sub("VOLUME \S+"," ",newstring)
    newstring=re.sub("CHAPTER \S+"," ",newstring)
    newstring=re.sub("\s\s+"," ",newstring)
    #return re.sub("([^\.|^ ])  +",r"\1 .  ",newstring).lstrip().rstrip()
    return newstring.lstrip().rstrip()

def display_tags(sentslist):
    taglist={}
    for s in sentslist:
        for t in s:
            form=t.lower_
            pos=t.pos_
            taglist[pos]=taglist.get(pos,0)+1
    print(len(taglist.keys()))
    print(taglist)
        
def get_train_test(sentslist,seed=candidateno):
    random.seed(seed)
    random.shuffle(sentslist)
    testsize=10
    train=[tag_sent(s) for s in sentslist[testsize:]]
    test=[tag_sent(s) for s in sentslist[:testsize]]
    return train,test
    
alice=clean_text(gutenberg.raw('carroll-alice.txt'))
nlp_alice=list(nlp(alice).sents)

NameError: name 'nlp' is not defined

The code below will generate (unique to you) training and test sets of pos-tagged sentences from the novel "Alice in Wonderland"

In [5]:
#do not change the code in this cell
allsents=list(nlp_alice)
train,test=get_train_test(allsents)

In [6]:
test

[[('this', 'DET'),
  ('was', 'VERB'),
  ('such', 'ADJ'),
  ('a', 'DET'),
  ('new', 'ADJ'),
  ('idea', 'NOUN'),
  ('to', 'ADP'),
  ('alice', 'PROPN'),
  (',', 'PUNCT'),
  ('that', 'ADP'),
  ('she', 'PRON'),
  ('was', 'VERB'),
  ('quite', 'ADV'),
  ('silent', 'ADJ'),
  ('for', 'ADP'),
  ('a', 'DET'),
  ('minute', 'NOUN'),
  ('or', 'CCONJ'),
  ('two', 'NUM'),
  (',', 'PUNCT'),
  ('which', 'ADJ'),
  ('gave', 'VERB'),
  ('the', 'DET'),
  ('pigeon', 'PROPN'),
  ('the', 'DET'),
  ('opportunity', 'NOUN'),
  ('of', 'ADP'),
  ('adding', 'VERB'),
  (',', 'PUNCT'),
  ("'", 'PUNCT'),
  ('you', 'PRON'),
  ("'re", 'VERB'),
  ('looking', 'VERB'),
  ('for', 'ADP'),
  ('eggs', 'NOUN'),
  (',', 'PUNCT'),
  ('i', 'PRON'),
  ('know', 'VERB'),
  ('that', 'DET'),
  ('well', 'ADV'),
  ('enough', 'ADV'),
  (';', 'PUNCT'),
  ('and', 'CCONJ'),
  ('what', 'NOUN'),
  ('does', 'VERB'),
  ('it', 'PRON'),
  ('matter', 'VERB'),
  ('to', 'ADP'),
  ('me', 'PRON'),
  ('whether', 'ADP'),
  ('you', 'PRON'),
  ("'re", 'VERB

In the cell below, a class `unigram_tagger` is defined, which will be used in parts a and b of this question

In [7]:
#do not change the code in this cell
class unigram_tagger():
    
    def __init__(self,trainingdata=[]):
        self.tagcounts={}
        self.wordcounts={}
        self.tagperwordcounts={}
        self.train(trainingdata=trainingdata)
        
    def train(self,trainingdata):
        
        for sentence in trainingdata:
            for token,tag in sentence:
                self.tagcounts[tag]=self.tagcounts.get(tag,0)+1
                self.wordcounts[token]=self.wordcounts.get(token,0)+1
                current=self.tagperwordcounts.get(token,{})
                current[tag]=current.get(tag,0)+1
                self.tagperwordcounts[token]=current
                

a) Create an instance of the `unigram_tagger` class which is trained on your training sentences from "Alice in Wonderland" (stored in `train`).
Explain what is stored, after training, in the three instance variables:
* `self.tagcounts`
* `self.wordcounts`
* `self.tagperwordcounts`

You should refer to the code and specific examples from the training data. \[4 marks \]

Instantiating the *unigram_tagger* class the training data

In [8]:
unigramTrainer=unigram_tagger(train) #  unigram tagger for the training data
# unigramTester=unigram_tagger(test) #  unigram tagger for the testing data

Next, get the `tagcounts` of the training data. `tagcounts` is a dictionary which counts the number of occurrences for each part of speech tag. The key of the dictionary is the POS tag and the value is the number of occurences of that POS. For example, `INTJ` occurs a total of 158 times, `PUNCT` (as in punctuation marks) occurs a total of 7162 times, it is the highest count, which makes sense because punctuation marks occur a lot of times in the English language, verbs occur 6295 times, etc...

In [9]:
tag_count=unigramTrainer.tagcounts
tag_count

{'INTJ': 158,
 'PUNCT': 7162,
 'VERB': 6195,
 'PROPN': 1395,
 'DET': 2853,
 'NOUN': 3649,
 'ADP': 2908,
 'PRON': 2809,
 'PART': 861,
 'CCONJ': 1117,
 'ADV': 2684,
 'ADJ': 2105,
 'NUM': 196,
 'X': 1}

Explanation of the code:

`self.tagcounts[tag]=self.tagcounts.get(tag,0)+1` where `tagcounts[tag]` stands for setting the dictionary's key as the part of speech tag and the `tagcounts.get(tag,0)+1` part will set the initial value for every POS tag as 0 and increment 1 everytime that specific tag is found.

If we look at the following, the key occuring the most frequently is `PUNCT`

In [10]:
max(tag_count,key=lambda k: tag_count[k])

'PUNCT'

Least occuring POS tag is `X`:

In [11]:
min(tag_count,key=lambda k: tag_count[k])

'X'

`wordcounts` is a dictionary with `key:value` pairs, key being a token in the training data, value being the total number of times it occurs.

In [12]:
word_count=unigramTrainer.wordcounts
word_count

{'yes': 12,
 ',': 2390,
 'please': 19,
 'do': 139,
 '!': 445,
 "'": 2200,
 'pleaded': 3,
 'alice': 394,
 '.': 964,
 'the': 1618,
 'cook': 13,
 'threw': 2,
 'a': 620,
 'frying': 1,
 '-': 138,
 'pan': 1,
 'after': 42,
 'her': 247,
 'as': 254,
 'she': 546,
 'went': 82,
 'out': 116,
 'but': 168,
 'it': 589,
 'just': 52,
 'missed': 2,
 'pig': 11,
 'and': 854,
 'pepper': 8,
 'i': 531,
 "'ve": 42,
 'something': 18,
 'important': 6,
 'to': 722,
 'say': 51,
 'then': 93,
 'you': 401,
 'should': 32,
 'what': 140,
 'mean': 10,
 'march': 34,
 'hare': 31,
 'on': 192,
 'would': 96,
 'not': 146,
 'could': 85,
 'join': 9,
 'dance': 13,
 'why': 39,
 "n't": 213,
 'anything': 20,
 'about': 94,
 'even': 19,
 'if': 95,
 'fell': 6,
 'off': 71,
 'top': 8,
 'of': 508,
 'house': 18,
 'ca': 28,
 'think': 50,
 'how': 68,
 'glad': 11,
 'am': 15,
 'see': 66,
 'again': 80,
 'dear': 29,
 'old': 18,
 'thing': 49,
 'remarked': 10,
 'answered': 4,
 'herself': 83,
 'asked': 17,
 'looked': 44,
 'down': 101,
 'at': 208,
 '

The most frequently occurring token is a comma, a punctuation mark, which makes sense because as stated earlier, the most frequently occuring POS tag was `PUNCT` for punctuation marks

In [13]:
max(word_count,key=lambda k: word_count[k])

','

Least occurring word was *frying*

In [14]:
min(word_count,key=lambda k: word_count[k])

'frying'

`self.wordcounts[token]=self.wordcounts.get(token,0)+1` means that `wordcounts` is a dictionary with token as the key, then it sets the initial value as `0` and increments `1` every time that specific token is found

`tagperwordcounts` take the tagged WSJ text and returns a dictionary containing the number of part of speech tags which each word type has << EX 1.2 week 8

b) In the cells below, **extend** the code for the `unigram_tagger` class so that it also has a `tag()` method.  This method should assign the tag, $t$, which maximises the unigram tag probability, $P(t|w)$, for the observed word, $w$.  **Evaluate** the performance of the `unigram_tagger` on your test sentences.  **Discuss** your results. \[8 marks\] 



In [15]:
import operator
#do not change the code in this cell
class unigram_tagger():
    def __init__(self,trainingdata=[]):
        self.tagcounts={}
        self.wordcounts={}
        self.tagperwordcounts={}
        self.train(trainingdata=trainingdata)
        
    def train(self,trainingdata):
        for sentence in trainingdata:
            for token,tag in sentence:
                self.tagcounts[tag]=self.tagcounts.get(tag,0)+1
                self.wordcounts[token]=self.wordcounts.get(token,0)+1
                current=self.tagperwordcounts.get(token,{})
                current[tag]=current.get(tag,0)+1
                self.tagperwordcounts[token]=current
    def back_off(self):
        #find which is the most frequent tag assigned to any type
        combined={}
        for adict in self.tagperwordcounts.values():
            for (tag,value) in adict.items():
                combined[tag]=combined.get(tag,0)+value
        ordered=sorted(list(combined.items()),key=operator.itemgetter(1),reverse=True)
        return ordered[0][0]
    def best_tag(self,word,default='N'):
        dist=self.tagperwordcounts.get(word,{default:1})
        ordered=sorted(list(dist.items()),key=operator.itemgetter(1),reverse=True)
        return ordered[0][0]
    def tag(self,words):
        default_tag=self.back_off()
        return [(word,self.best_tag(word)) for word in words]
    def evaluate_uni_pos_tag(self,goldstandard):
        goldtoks,goldtags=zip(*goldstandard)
        pretoks,pretags=zip(*self.tag(goldtoks))
        print(goldtags[:10])
        print(pretags[:10])
        n=len(pretags)
        correct=0
        for (pre,gold) in zip(pretags,goldtags):
            if pre==gold:
                correct+=1
        return correct/n

In [16]:
unigramTrainer2=unigram_tagger(train)

In [17]:
def evaluate_all(train_data,test_data):
    unigram=unigram_tagger(train)
    performance=[]
    for sentences in test_data:
        performance.append(unigram.evaluate_uni_pos_tag(sentences))
    return performance
evaluate_all(train,test)

('DET', 'VERB', 'ADJ', 'DET', 'ADJ', 'NOUN', 'ADP', 'PROPN', 'PUNCT', 'ADP')
('DET', 'VERB', 'ADJ', 'DET', 'ADJ', 'NOUN', 'PART', 'PROPN', 'PUNCT', 'ADP')
('DET', 'NOUN', 'ADJ', 'ADP', 'DET', 'PROPN', 'ADJ', 'PART', 'VERB', 'DET')
('DET', 'N', 'ADJ', 'ADP', 'DET', 'NOUN', 'ADJ', 'PART', 'VERB', 'DET')
('ADP', 'DET', 'VERB', 'DET', 'ADJ', 'NOUN', 'PUNCT', 'ADV', 'VERB', 'NUM')
('ADP', 'DET', 'VERB', 'DET', 'ADJ', 'NOUN', 'PUNCT', 'ADV', 'VERB', 'NUM')
('DET', 'ADP', 'DET', 'NOUN', 'VERB', 'PART', 'ADP', 'ADV', 'PUNCT', 'NUM')
('DET', 'ADP', 'DET', 'NOUN', 'VERB', 'PART', 'ADP', 'ADV', 'PUNCT', 'NUM')
('ADV', 'PUNCT', 'ADV', 'VERB', 'ADV', 'NOUN', 'ADP', 'PRON', 'PUNCT', 'CCONJ')
('ADV', 'PUNCT', 'ADV', 'VERB', 'ADV', 'NOUN', 'ADP', 'PRON', 'PUNCT', 'CCONJ')
('ADP', 'PRON', 'VERB', 'DET', 'PRON', 'VERB', 'PART', 'ADP', 'ADJ', 'NOUN')
('ADP', 'PRON', 'VERB', 'DET', 'PRON', 'VERB', 'PART', 'ADP', 'ADJ', 'NOUN')
('CCONJ', 'PRON', 'VERB', 'DET', 'NOUN', 'CCONJ', 'VERB', 'ADV', 'PUNCT')
('CCO

[0.9180327868852459,
 0.7692307692307693,
 0.8571428571428571,
 0.925,
 0.9047619047619048,
 0.9444444444444444,
 0.8888888888888888,
 0.8461538461538461,
 0.8571428571428571,
 0.9130434782608695]

In the cell below, a `hmm_tagger` class is defined, which will be used in parts c and d of this question.

In [18]:
#do not change the code in this cell
class hmm_tagger():
    
    def __init__(self,trainingdata=[]):
        
        self.emissions={}
        self.transitions={}
        self.train(trainingdata=trainingdata)
        
    def train(self,trainingdata):
        
        for sentence in trainingdata:
            previous="START"
            for token,tag in sentence:
                
                current=self.emissions.get(tag,{})
                current[token]=current.get(token,0)+1
                self.emissions[tag]=current
                bigram=self.transitions.get(previous,{})
                bigram[tag]=bigram.get(tag,0)+1
                self.transitions[previous]=bigram
                previous=tag
                
        self.emissions={tag:{token:freq/sum(countdict.values()) for (token,freq) in countdict.items()}for (tag,countdict) in self.emissions.items()}
        self.transitions={tag:{token:freq/sum(countdict.values()) for (token,freq) in countdict.items()}for (tag,countdict) in self.transitions.items()}

c) **Create** an instance of the `hmm_tagger` class which is trained on your training sentences from "Alice in Wonderland" (stored in `train`).  With reference to your testing sentences (stored in `test`), **explain**
* how to calculate the probability of an observed sequence of words for a given sequence of tags: $$P(w_1^n|t_1^n)$$
* how to calculate the probablity of a possible sequence of tags for a given sequence of words: $$P(t_1^n|w_1^n)$$

\[6 marks\]

Calculating the probability of a word sequence given the tag sequence: $P(w_1^n|t_1^n)$. Breaking it down, it means calculating the probability of a word sequence $w_1^n$, $1$ being the first word until $n$ which is the number of words, so from the first word **until** the last word $n$. Essentially, it means going through all the words in the tag sequence. Same for $t_1^n$, which goes through the tags from $1$ until $n$. So since it is a **conditional probability**, it just means the **total** probability of all words given their tag, all until $n$ which is the total length of the words given.

To calculate it, we need the probability of **each** word given its tag, which is $P(w_{i}|t_{i})$, and then find the total product of them from $i$ to $n$. We find the product and not the summation because we are trying to find $P(\text {taggedWord}_{1} \text{ and taggedWord}_{2}\text{ and.....taggedWord}_{i})$, basically the probability of those words occuring together with each word having their specific tag. Thus we know that $P(A \thinspace and \thinspace B) = P(A) \times P(B)$ according to the probability of independent events.

Thus, to find the total probability of $P(w_1^n|t_1^n)$ we must multiply all probability together for each word given its tag, giving us the following rule: $$P(w_1^n|t_1^n) = \prod_{i}^{n} P(w_{i}|t_{i})$$ this rule is known as the Forward Algorithm

First, create an instance of the `hmm_tagger` and train it on our train data:

In [19]:
hmm_train=hmm_tagger(train)

Next, let's get an example of a sentence from our test data... let's get six tagged words from index[0]. It has tuples `(word,tag)` in a list

In [20]:
test[0][:6]

[('this', 'DET'),
 ('was', 'VERB'),
 ('such', 'ADJ'),
 ('a', 'DET'),
 ('new', 'ADJ'),
 ('idea', 'NOUN')]

We have the sentence 'this was such a new idea', with each word's tag.<br/>
$$this \,\to\, DET$$
$$was \,\to\, VERB$$
$$such \,\to\, ADJ$$ 
$$a \,\to\, DET$$ 
$$new \,\to\, ADJ$$ 
$$idea \,\to\, NOUN$$ 

We must calculate the probability of each word given its tag. So we must calculate the probability of 'this' being a `DET`, 'was' being a `VERB`, 'such' being an `ADJ`, 'a' being a `DET`, 'new' being an `ADJ`, and 'idea' being a `NOUN`... these are called *emissions* (emissions is the probability of a word given a tag) and we can get them from the *trained hmm_tagger*

In [21]:
hmm_train.emissions

{'INTJ': {'yes': 0.0759493670886076,
  'please': 0.0759493670886076,
  'no': 0.12658227848101267,
  'oh': 0.27848101265822783,
  'well': 0.15822784810126583,
  'ah': 0.03164556962025317,
  'dear': 0.06962025316455696,
  'sh': 0.006329113924050633,
  'hm': 0.006329113924050633,
  'sir--': 0.006329113924050633,
  'nay': 0.006329113924050633,
  'wow': 0.0379746835443038,
  'ugh': 0.012658227848101266,
  'o': 0.0189873417721519,
  'like': 0.006329113924050633,
  'sure': 0.02531645569620253,
  'alas': 0.02531645569620253,
  'right': 0.012658227848101266,
  '_': 0.006329113924050633,
  'eh': 0.006329113924050633,
  'shy': 0.006329113924050633},
 'PUNCT': {',': 0.33370566880759567,
  '!': 0.062133482267523035,
  "'": 0.3064786372521642,
  '.': 0.1345992739458252,
  '-': 0.01926836079307456,
  '--': 0.018570231778832727,
  ':': 0.029740296006702038,
  '?': 0.027506283161128176,
  '(': 0.007679419156660151,
  ')': 0.007260541748115051,
  ';': 0.02652890254118961,
  '"': 0.014241831890533371,
  

So now, using the emissions given, to calculate $P(w_1^n|t_1^n) = \prod_{i}^{n} P(w_{i}|t_{i})$ for the given sentence, where $n=6$...$$P(this|DET) \times P(was|VERB) \times P(such|ADJ) \times P(a|DET) \times P(new|ADJ) \times P(idea|NOUN)$$ To get these, extract the emission for each one of them and multiply them together.

In [22]:
emissions=hmm_train.emissions
emission=emissions["DET"]["this"]*emissions["VERB"]["was"]*emissions["ADJ"]["such"]*emissions["DET"]["a"]*emissions["ADJ"]["new"]*emissions["NOUN"]["idea"]
emission

6.01215233760472e-11

It is a very small value in scientific notation which stands for $6.01215233760472 × 10^{-11}$ because the probability of those words occuring together in that specific sequence with those specific tags is not that common...

Here is a function that calculates the total probability for any index of test data you want

In [23]:
hmm=hmm_tagger(train)
emissions=hmm_train.emissions
def forward_algorithm(test,emissions):
    result=1
    someList=[]
    for word1,tag1 in test:
        for tag2,dictionary in emissions.items():
            if tag1==tag2:
                for word2,emission in dictionary.items():
                    if word1==word2:
                        #someList.append((dictionary[word1]))
                        x=dictionary[word1]
                        result*=x                 
    return result
forward_algorithm(test[2],emissions)
    


1.716118011323917e-51

Last but not least, to calculate the probability of a tag sequence given a sequence of tag words $P(t_1^n|w_1^n)$ where $1$ is the first tag/word and $n$ is the length of the tags/words as discussed earlier, so it goes through the specific tags given their specific word for each one.

If we apply Bayes' theorem which is: $ P(A \mid B) = \frac{P(B \mid A) \, P(A)}{P(B)} $ then we can conclude that...<br/>
$$P(t_1^n|w_1^n) = \frac{P(w_1^n|t_1^n) \, P(t_1^n)}{P(w_1^n)}$$

Simply drop the denominator $P(w_1^n)$ because we are interested which value of $t_1^n$ is more probable for classification and $P(t_1^n)$ is a probability $\therefore$ it is positive and can be dropped, leaving us with...$$P(t_1^n|w_1^n) = P(w_1^n|t_1^n) \, P(t_1^n)$$

A first-order hidden Markov model performs the Markov process with hidden states by instantianting two assumptions, the _output independence_ assumption and the _Markov assumption_...

Assume _output independence_, which means the probability of an output observation $w_1^n$ depends only and only on the current state which produced the observation $t_1^n \therefore$ $\text{Output Independence}\text : P(o_{i}|q_{1}...q_{i},q_{T},o_{1},...,o_{i},o_{T})=P(o_{i}|q_{i}) \therefore P(t_1^n|w_1^n) = P(w_i|t_i) \, P(t_1^n)$

Now for the _Markov assumption_ to simplify $P(t_1^n)$, using the first-order Markov chain which deals with bigrams, states that the probability of some state depends on the previous state... this concludes that the current tag depends on the previous tag: $P(q_i|q_1...q_i)=P(q_i|q_{i-1}) \therefore P(t_1^n)=P(t_i|t_{i-1})$ and this is known as `transitions` which are the bigram probabilities. Basically, the probability of a tag being what it is given its previous tag.

Finally leaving us with an overall equation of: $$P(t_1^n|w_1^n) = P(w_i|t_i)\thinspace P(t_i|t_{i-1})$$ where $P(w_i|t_i)$ are the `emissions` we calculated earlier and $P(t_i|t_{i-1})$ are the `transitions`

$\therefore$ just do $\text{transitions} \thinspace \times \text{emissions}$

Possible code that can be done to calculate $\text{transitions} \thinspace \times \text{emissions}$

In [24]:
def seq_tag_prob(test,emissions,transitions):
    result=1
    someList=[]
    for word1,tag1 in test:
        for tag2,dictionary in emissions.items():
            for word3,dictionary3 in transitions.items():
                for tag3,dictionary4 in dictionary3.items():
                    if tag1==tag2==tag3:
                        for word2,emission in dictionary.items():
                            if word1==word2:
                                #someList.append((dictionary[word1]))
                                x=dictionary[word1]
                                y=dictionary3[tag3]
                                result*=x 
    r=result*y
                                
    return r
x=hmm.transitions
y=hmm.emissions
seq_tag_prob(test[1],y,x)

1.1498080729991932e-183

For example, let's get the same sentence in the test data:

In [25]:
test[0][:6]

[('this', 'DET'),
 ('was', 'VERB'),
 ('such', 'ADJ'),
 ('a', 'DET'),
 ('new', 'ADJ'),
 ('idea', 'NOUN')]

We calculated the emission for this earlier = $6.01215233760472 × 10^{-11}$. Now we get the total transition for it.

In [26]:
transition=hmm.transitions["START"]["DET"]*hmm.transitions["DET"]["VERB"]*hmm.transitions["VERB"]["ADJ"]*hmm.transitions["ADJ"]["DET"]*hmm.transitions["DET"]["ADJ"]*hmm.transitions["ADJ"]["NOUN"]
transition

1.4307273770841426e-06

Now just multiply them together...

In [27]:
total=emission*transition
total

8.601750944611498e-17

In [28]:
hmm.transitions

{'START': {'INTJ': 0.0615748963883955,
  'DET': 0.10420367081113084,
  'PROPN': 0.07519242155121374,
  'PRON': 0.21314387211367672,
  'ADV': 0.14328004736530492,
  'VERB': 0.12492599171107165,
  'NOUN': 0.04854943753700414,
  'NUM': 0.004736530491415038,
  'CCONJ': 0.0674955595026643,
  'ADP': 0.047365304914150384,
  'PUNCT': 0.08762581409117821,
  'ADJ': 0.020722320899940794,
  'PART': 0.0011841326228537595},
 'INTJ': {'PUNCT': 0.8451612903225807,
  'VERB': 0.03870967741935484,
  'ADJ': 0.03225806451612903,
  'INTJ': 0.05161290322580645,
  'NOUN': 0.012903225806451613,
  'ADV': 0.0064516129032258064,
  'ADP': 0.0064516129032258064,
  'PROPN': 0.0064516129032258064},
 'PUNCT': {'INTJ': 0.007040982126737678,
  'PUNCT': 0.39465607510380935,
  'VERB': 0.11428055605705001,
  'NOUN': 0.04170427875067702,
  'CCONJ': 0.1357645784437624,
  'DET': 0.037551904675934285,
  'PRON': 0.09478245170608413,
  'ADV': 0.05398086297165553,
  'PROPN': 0.017873262321718722,
  'ADP': 0.07077089727387616,
  '

d) Using one of your `test` sentences as an example, **explain** how the Viterbi algorithm can be used to find the most likely tag sequence.  You do not need to write code for this question but should explain the calculations that need to be made at each step.  **Comment** on whether the sequence found by the Viterbi algorithm is correct for your chosen test sentence.  **Discuss** why using the Viterbi algorithm is better than the brute-force approach of enumerating and testing all tag sequence possibilities.\[7 marks\]


Finding the most likely tag sequence using brute force would be time-consuming. The number of possible tag sequences = $k^n$ such that $k = \text{number of tags, }n=\text{number of word tokens}$. For example, if we have the sentence "I have an idea" and we have the possible tagset [noun,verb,pronoun], then we have $k=3$ and $n=4$ giving us $3^4=81$ possible tag sequences. Although a bit time-consuming, it is still possible to try every one. However, imagine a tagset of size 10 and a sentence of length 10, that is $10^10=10000000000$ possible tag sequences...that would take days according to the rate of brute force. This yields an exponential complexity $O(k^n)$ 

The Viterbi algorithm is a classic example of dynamic programming - the HMM can discriminate highly ambiguous situations by using a _decoder_ by finding the most likely sequence that has observations where the search space of possible sequences has a time complexity of $O(n^t)$ which is too large for a brute force search - as mentioned earlier, for the first-order HMM we need the previous state, we just calculate the probability for each state and then memoise the probability. This gives rise to a linear complexity which is better than an exponential complexity. Essentially, this algorithm deals with the divide-and-conquer method:

Divide: recursively divide your problem into smaller sub-problems.<br/>
Conquer: solve each sub-problem and keep track of each solution<br/>
Combine: combine your solutions to the subproblems to finally get the solution for the big problem.</br>

Suppose we have an input sequence of words $w^n_1$. Our sub-problem will be for each word from $1$ to $n$ which will be a pair $(i,t)$ where $i= \text{position in the sequence and } i<n$, $t = \text{ part of speech tag}.$ 


For the first step, we solve for the first word in the word sequence $w_1^n$.<br/>
We use the notation $V(i,t)$ for each sub-problem.<br/>
Starting with the first word, we calculate the probability of the word given the first tag in the tag sequence $t_1^n$, then multiply it by the probability of that tag given the previous word.... so it would look something like this:<br/>
$V(1,t_1)=P(w_1|t_1) \times P(t_1|start)$ and then we do the same for that word using the other tags in the tagset.<br/>
This is where the recursive step comes in - once you have the solution for the first sub-problem, we can use this formula: $V(i,t)=max_{t' \in T}(V(i-1,t') \times P(t|t') \times P(w_i|t))$<br/>
Now for the second step, $V(position=2,tag)$, the previous tag could have been anything. Therefore, if it was a noun for example, then we have to calculate all the probabilities assuming that the word could be any tag and recursively do it for all the tagset. To clarify, say we have the tagset [noun,verb]. Therefore, assuming that the current word is a noun first, the previous tag could have been anything, so if it was a noun then it would look something like this: $P(t_2=N)=V(2-1=1,t_1=N) \times P(N|N) \times P(word_2|N)$. Assuming that the word is still a noun, we now calculate the total probability assuming the previous tag was a verb instead of a noun: $P(t_2=N)=V(2-1=1,t_1=V) \times P(N|V) \times P(word_2|N)$. Once calculated for all tagsets for that specific word being a specific tag, get the maximum probability out of all of them to get the $V(2,N)$.<br/>
Next, still working on $word_2$, we assume that it is a verb now. Then we do the same calculations, assuming that the previous tag could be a noun or a verb until all of the sub-problems are solved.

To simplify, for decoding we need to find *one* possibility for each hidden state state sequence and pick the highest probability. However, this is $O(N^T)$ complexity and is very time-consuming and that is why we resort to the Viterbi algorithm which is a dynamic programming algorithm that uses a similar trellis to the forward algorithm we discussed earlier. We have to compute the joint probability of the observation sequence with the best state sequence. The process is from *left to right*

A possible pseudocode for the Viterbi algorithm:


> def viterbi(hmm,test): <br/>
> > startTag="START"<br/>
> > current=[startTag]<br/>
> > for word,tag in test:<br/>
> > > prev=current<br/>
> > > current=[]<br/>
> > > find all possible tag for word using wordpertagcounts<br/>
> > > for each tag of possible tags:<br/>
> > > > find the highest wordpertagcounts<br/>
> > > > append it to current<br/>
> > return highest wordpertagcounts ending with tag "END"<br/>

Let's apply the Viterbi algorithm to our test data... I only use 3 words from my test data of index[3] just to keep this notebook short

In [36]:
my_test_data=test[3][:3]
my_test_data

[('some', 'DET'), ('of', 'ADP'), ('the', 'DET')]

<img src="tags.png">

We have the tags `DET`, `ADP`, `DET`, respectively. This gives us a total of 2 tagsets, `DET` and `ADP`. The length of the sentence is 3 words. $\therefore |tagsets|=2, |tokens|=3$

Viterbi initialisation: our base case. As discussed earlier, we have the pair $(i,t) \text{ where } i=\text{position in the sequence, } t=\text{current pos tag}$

<img src="vit1.png">

$\text{Sub-problem 1: assuming that 'some' is DET: }t_1=DET\\
V(1,DET)=P(t_1=DET)=P(some|DET) \times P(DET|start)=\thinspace ...\\
\text{we must find the probability of 'some' given that it's DET and the probability of DET given it's the starting tag,}\\ \text{therefore we need to find the emission for P(some|DET) and transition for P(DET|start):}
$


In [87]:
hmm=hmm_tagger(train)
someDET=hmm.emissions["DET"]["some"]
startDET=hmm.transitions["START"]["DET"]
vit1=someDET*startDET
vit1

0.0018262122469528714

$\therefore V(1,DET)=P(t_1=DET)=P(some|DET) \times P(DET|start)=0.0018262122469528714$

$\text{Sub-problem 2: assuming that 'some' is ADP: }t_1=ADP\\
V(1,ADP)=P(t_1=ADP)=P(some|ADP) \times P(ADP|start)=\thinspace ...\\
\text{we must find the probability of 'some' given that it's ADP and the probability of ADP given it's the starting tag,}\\ \text{therefore we need to find the emission for P(some|ADP) and transition for P(ADP|start):}
$

In [77]:
hmm=hmm_tagger(train)
hmm.emissions["ADP"]["some"]*hmm.transitions["START"]["ADP"]

KeyError: 'some'

We got an error because 'some' does not exist in the ADP tags, meaning it has a 0 probability of being an adposition, and that makes sense because an adposition is for prepositions and postpositions, and 'some' obviously isn't. Look at the dictionary below for adpositons. All of the words you can see are prepositions and postpositions, and 'some' is not found anywhere in that dictionary, therefore we will assume it has a probability of 0.

In [79]:
hmm.emissions["ADP"]

{'after': 0.012379642365887207,
 'as': 0.06946354883081156,
 'about': 0.029917469050894086,
 'if': 0.03266850068775791,
 'off': 0.004814305364511692,
 'of': 0.17125171939477304,
 'at': 0.06327372764786796,
 'before': 0.006533700137551582,
 'to': 0.09800550206327373,
 'in': 0.11863823933975241,
 'by': 0.019601100412654747,
 'for': 0.05055020632737277,
 'into': 0.023039889958734527,
 'near': 0.0041265474552957355,
 'like': 0.017537826685006877,
 'on': 0.0374828060522696,
 'with': 0.061210453920220086,
 'whether': 0.0034387895460797797,
 'though': 0.002751031636863824,
 'that': 0.04986244841815681,
 'so': 0.0020632737276478678,
 'except': 0.001375515818431912,
 'under': 0.005502063273727648,
 'till': 0.007221458046767538,
 'from': 0.012379642365887207,
 'through': 0.004470426409903714,
 'since': 0.001375515818431912,
 'along': 0.0010316368638239339,
 'without': 0.008940852819807428,
 'down': 0.003782668500687758,
 'round': 0.002407152682255846,
 'out': 0.013411279229711142,
 'upon': 0.008

$\therefore V(1,ADP)=P(t_1=ADP)=P(some|ADP) \times P(ADP|start)=0$

Now that was our base case, this is where the recursive step comes as discussed earlier this is the equation we need to use now:$V(i,t)=max_{t' \in T}(V(i-1,t') \times P(t|t') \times P(w_i|t))$<br/>
Step 2:<img src="vit2.png">
$\text{Sub-problem 1: assuming that 'of' is DET: }t_2=DET\\
\text{the previous tag for 'some' is obviously DET and not ADP as the probability for ADP was 0}\\
\text{if it was DET:}\\
V(2,DET)=P(t_2=DET)=V(1,DET) \times P(DET|DET) \times P(of|DET)=\thinspace ...\\
$



In [84]:
hmm.emissions["DET"]["of"] # no probability of the word 'of' being a DET so probability = 0

KeyError: 'of'

$V(2,DET)=P(t_2=DET)=V(1,DET) \times P(DET|DET) \times P(of|DET)=0$

And now assuming that the previous tag was ADP would be pointless because it is gonna give us a probability of 0 as well. So our max is the first one we calculated.<br/>
So now we move on to sub-problem 2, assuming 'of' is ADP.<br/>
If previous tag was DET: $P(t_2=ADP)=V(1,DET) \times P(ADP|DET) \times P(of|ADP)$

In [88]:
vit1*hmm.transitions["DET"]["ADP"]*emissions["ADP"]["of"]

2.9617796058585477e-06

$P(t_2=ADP)=V(1,DET) \times P(ADP|DET) \times P(of|ADP)=2.9617796058585477e-6=0.0000029617796058585477$

If previous tag was ADP: $P(t_2=ADP)=0$ so our max was $0.0000029617796058585477$

Now we do the same for the $V(3,DET)$ and $V(3,ADP)$ following the same steps and getting the maximum probability, which will be our optimal path.

In [89]:
import io
from IPython.nbformat import current

with io.open("a3_1.ipynb", 'r', encoding='utf-8') as f:
    nb = current.read(f, 'json')

word_count = 0
for cell in nb.worksheets[0].cells:
    if cell.cell_type == "markdown":
        word_count += len(cell['source'].replace('#', '').lstrip().split(' '))
print(word_count)

2556
